In [ ]:
from fact.io import read_h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
test = read_h5py("data/gamma_test_dl3.hdf5", key="events")
crab = read_h5py("data/open_crab_sample_dl3.hdf5", key="events")

gamma_test = test["gamma_prediction"] >= 0.8
gamma_crab = crab["gamma_prediction"] >= 0.8

theta2_s = crab["theta_deg"][gamma_crab].to_numpy()**2
theta2_bg = crab[["theta_deg_off_1","theta_deg_off_2","theta_deg_off_3","theta_deg_off_4","theta_deg_off_5"]][gamma_crab].to_numpy().flatten()**2
print(theta2_bg)

In [ ]:
hist_s = np.histogram(theta2_s[theta2_s<=0.025], bins=20)
hist_bg = np.histogram(theta2_bg[theta2_bg<=0.025], bins=20)
print(hist_s)

fig, ax = plt.subplots(constrained_layout=True)

#ax.bar(hist_s[1][:-1],hist_s[0])
#ax.bar(hist_bg[1][:-1],hist_bg[0]/5)

ax.hist(theta2_s[theta2_s<=0.025], bins=20, histtype="step")
ax.hist(theta2_bg[theta2_bg<=0.025], bins=20, histtype="step")


plt.show()

In [ ]:
N_on = len(theta2_s[theta2_s<=0.025])
N_off = len(theta2_bg[theta2_bg<=0.025])
print(N_on,N_off)

alpha = 0.2

S = np.sqrt(2) * np.sqrt(N_on * np.log( (1+alpha)/alpha * N_on/(N_on + N_off)) + N_off * np.log((1+alpha) * N_off/(N_on + N_off)))
S

In [ ]:
E_true = test["corsika_event_header_total_energy"][gamma_test].to_numpy()
E_pred = test["gamma_energy_prediction"][gamma_test].to_numpy()

hist_E_true, bins_E_true = np.histogram(E_true, np.logspace(2.7,4.2,20))
hist_E_pred, bins_E_pred = np.histogram(E_pred, np.logspace(2.7,4.2,30))
print(hist_E_true,hist_E_pred)


In [ ]:
A = np.outer(1/hist_E_true,hist_E_pred)
g = hist_E_true@A

fig, ax = plt.subplots(constrained_layout=True)

ax.stairs(hist_E_true,bins_E_true,label="True")
ax.stairs(hist_E_pred,bins_E_pred,label="Predicted")
ax.stairs(g/19,bins_E_pred,label="with A")

ax.set_xscale("log")
ax.legend(loc="best")

plt.show()

In [ ]:
A = np.outer(hist_E_pred, 1/hist_E_true)
g = A@hist_E_true
print(np.isclose(g/19,hist_E_pred))
